In [1]:
from datasets import config
print(config.HF_DATASETS_CACHE)

/home/quanvo/.cache/huggingface/datasets


## PROCESS DATA FOR FINAL EVALUATION<br>
In The Vault dataset, there are hundred thousands to millions samples for each programming language. Evaluate LLMs on all samples are prohibitedly costly. To compare different methods with each others, some hundreds (200 per language in LLM4CodeSummarization) are enough.
In this project, I want to answer: <br>
    <b>RQ1: How effective are in-context learning using LLMs in code comment generation? <br>
	RQ2: Does project-specific context improve the performance of LLMs? <br>
	RQ3: Does retrieval-augmented generation improve the performance of LLMs? <br>
	RQ4: How do LLMs perform for different programming languages? <br></b>
To evaluate LLMs for RQ1, RQ2 and RQ3, I collect all functions from 100 different projects, each project needs to have at least 5 functions in The Vault to be considered.
data structure look like this: <br>
<b>ProjectObject:</b><br>
    {
    "project_name": name of the project, corresponding to "repo" in The Vault, <br>
    "functions": List[FunctionObject]. List of functions belongs to this project<br>
    } <br>
<b>FunctionObject</b>: <br>
    {
        "code": code of function, corresponding to "code" in The Vault, <br>
        "docstring": full docstring of function, corresponding to "docstring" in The Vault, <br>
        "project": name of the project, corresponding to "repo" in The Vault, <br>
        "code_tokens": list of tokens, corresponding to "code_tokens" in The Vault, <br>
        "code_tokens_processed": list of tokens after some processing for bm25 retrieval, <br>
        "bm25": list of code and docstring of 5 most similar functions retrieved using bm25, <br>
        "CodeBERT": list of code and docstring of 5 most similar functions retrieved using CodeBERT embedding, <br>
    }

<b>STEP 1: Collect functions from 200 projects</b>

In [2]:
from datasets import load_dataset
from collections import defaultdict
import json
import os

def the_vault_function_to_json(
    split_set="test",
    languages=["java"],
    streaming=True,
    num_proj=None,
    num_method=None,
    min_num_functions=5, #keep only project with at least min_num_functions functions
    write_to_file=False,
    output_dir="/vault_function_100_proj_json"
):
    """
    Loads and processes functions from The Vault dataset, grouping them by project.

    This function fetches code samples from the `Fsoft-AIC/the-vault-function` dataset
    using the Hugging Face `datasets` library. It groups functions by project and filters 
    them based on a minimum number of functions per project. The output can optionally 
    be saved to disk as JSONL files.

    Args:
        split_set (str): Dataset split to load (e.g., "train", "train/small", "train/medium").
        languages (List[str]): List of programming languages to load (e.g., ["java"]).
        streaming (bool): Whether to stream the dataset instead of loading fully into memory.
        num_proj (int, optional): Maximum number of unique projects to collect. Cannot be used with `num_method`.
        num_method (int, optional): Maximum number of individual methods/functions to collect. Cannot be used with `num_proj`.
        min_num_functions (int): Minimum number of functions a project must have to be included.
        write_to_file (bool): Whether to write the processed data to JSONL files.
        output_dir (str): Directory where JSONL files will be saved if `write_to_file` is True.

    Returns:
        dict: A dictionary mapping each language to a list of project entries. 
              Each entry is a dict with keys `"project_name"` and `"functions"`, 
              where `"functions"` is a list of function metadata.
              
    Raises:
        AssertionError: If both `num_proj` and `num_method` are provided.
    """
    
    assert not (num_proj and num_method), "Cannot use both num_proj and num_method"

    if write_to_file:
        os.makedirs(output_dir, exist_ok=True)

    data_lang = {}
    for lang in languages:
        dataset = load_dataset("Fsoft-AIC/the-vault-function", 
                               split_set=[split_set], 
                               languages=[lang], 
                               streaming=False,
                               trust_remote_code=True)
        if split_set == "train/small":
            split_set = "train_small"
        elif split_set == "train/medium":
            split_set = "train_medium"
        dataset = dataset[split_set]
        print(f"Loaded {lang}, split={split_set}, streaming={streaming}")

        data = []
        project_function = []
        project = ""
        proj_counts = defaultdict(int)
        seen_projects = set()
        i = 0
        for sample in dataset:
            i += 1
            #print(f"{i}: {len(seen_projects)}")
            if num_proj and len(seen_projects) >= num_proj:
                break
            if num_method and len(data) >= num_method:
                break
            item = {
                "code": sample["code"],
                "code_tokens": sample["code_tokens"],
                "docstring": sample["docstring"],
                "project": sample["repo"],
                "name": sample["identifier"]
            }
            # Encounter new project: append project_function to data, refresh project_function
            if project != item["project"] and project != "":
                if len(project_function) > min_num_functions:
                    data.append({"project_name": project, "functions": project_function})
                    if num_proj:
                        seen_projects.add(project)
                project_function = []
            project = item["project"]
            project_function.append(item)
            proj_counts[project] += 1

        print(f"{lang} - collected {i} samples across {len(seen_projects)} projects")

        if write_to_file:
            output_path = os.path.join(output_dir, f"{lang}_{split_set}.jsonl")
            with open(output_path, "w") as f:
                for item in data:
                    f.write(json.dumps(item) + "\n")
            print(f"Saved to {output_path}")
        data_lang[lang] = data
    return data_lang

In [3]:
data_lang = the_vault_function_to_json(split_set="test", 
                                       languages=["java"], 
                                       num_proj=200, 
                                       write_to_file=False)
data = data_lang["java"]

Loaded java, split=test, streaming=True
java - collected 7713 samples across 200 projects


<b>Step 2: Process code tokens </b>

In [4]:
import re

def split_identifier(token):
    # Split snake_case and then camelCase
    parts = re.split(r'[_]', token)
    split_parts = []
    for part in parts:
        # Split camelCase and PascalCase using lookahead and lookbehind
        camel_parts = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?![a-z])', part)
        split_parts.extend(camel_parts)
    return split_parts

def process_tokens(tokens):
    split_tokens = []
    for token in tokens:
        if re.match(r'^[A-Za-z_]+$', token):
            split_tokens.extend(split_identifier(token))
        else:
            split_tokens.append(token)
    split_tokens = [t for t in split_tokens if not is_stopword(t)]
    return split_tokens

JAVA_KEYWORDS = {
    'abstract', 'assert', 'boolean', 'break', 'byte', 'case', 'catch', 'char',
    'class', 'const', 'continue', 'default', 'do', 'double', 'else', 'enum',
    'extends', 'final', 'finally', 'float', 'for', 'goto', 'if', 'implements',
    'import', 'instanceof', 'int', 'interface', 'long', 'native', 'new',
    'package', 'private', 'protected', 'public', 'return', 'short', 'static',
    'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'throws',
    'transient', 'try', 'void', 'volatile', 'while', 'true', 'false', 'null'
}

JAVA_API_COMMON = {
    'System', 'out', 'in', 'err', 'print', 'println', 'printf', 'String', 'Integer',
    'List', 'ArrayList', 'Map', 'HashMap', 'Set', 'HashSet', 'Math', 'Object',
}

PUNCTUATIONS = ["''", "'", "``", "`", "-LRB-", "-RRB-", "-LCB-", "-RCB-", ".", "?", "!", ",", ":", "-", "--", "...", ";"]

def is_stopword(token):
    token_lower = token.lower()
    return (token in JAVA_KEYWORDS or token in JAVA_API_COMMON or token in PUNCTUATIONS)

In [5]:
for i in range(0, len(data)):
    for j in range(0, len(data[i]['functions'])):
        tokens = data[i]['functions'][j]['code_tokens']
        data[i]['functions'][j]['code_tokens_processed'] = process_tokens(tokens)

<b>Step 3: run bm25 </b> <br>
In this step, I use bm25s (https://github.com/xhluca/bm25s/tree/main), for faster bm25. However, the retrieve() method of bm25s does not return indices of retrieved documents, so we cannot find corresponding docstring. Therefore, I have to modify the code. In bm25s/bm25s/__init__.py file,  change "allowed_return_as = ["tuple", "documents"]" on line 685 to "allowed_return_as = ["tuple", "documents", "tuple_custom", "documents_custom"]". On lines 853-858, insert: <br>
elif return_as == "tuple_custom": <br>
    return Results(documents=retrieved_docs, scores=scores), indices <br>
elif return_as == "documents_custom": <br>
    return retrieved_docs, indices <br>
Then, I use indices to get docstring. Below is the code example.

In [6]:
import bm25s
dataset = load_dataset("Fsoft-AIC/the-vault-function", 
                       split_set=["train/small"], 
                       languages=["java"], 
                       streaming=False,
                       trust_remote_code=True)

# UNCOMMENTED TO RUN
print("create corpus")
corpus = [sample["code"] for sample in dataset["train_small"]]
corpus_tokens = [process_tokens(sample["code_tokens"]) for sample in dataset["train_small"]]

print("indexing...")
retriever = bm25s.BM25(corpus=corpus)
retriever.index(corpus_tokens)

# query_tokens = data[0]['functions'][5]['code_tokens_processed']
# res, indices = retriever.retrieve([query_tokens], k=5, return_as="tuple_custom")
# docs = res.documents
# scores = res.scores
# print(f"Best result (score: {scores[0, 4]:.2f}): {docs[0, 4]}, index: {indices[0, 4]}")

# UNCOMMENTED TO RUN
for i in range(len(data)):
    for j in range(len(data[i]['functions'])):
        print(f"{i}: {j}")
        query_tokens = data[i]['functions'][j]['code_tokens_processed']
        res, indices = retriever.retrieve([query_tokens], k=5, return_as="tuple_custom")
        indices = indices[0, 0:]
        docs = res.documents
        scores = res.scores
        retrieved_examples = [{'code': dataset['train_small'][ind.item()]['code'],
                               'docstring': dataset['train_small'][ind.item()]['docstring'],
                               'code_tokens': dataset['train_small'][ind.item()]['code_tokens'],} for ind in indices]
        data[i]['functions'][j]['bm25'] = retrieved_examples

create corpus
indexing...


BM25S Create Vocab:   0%|          | 0/351213 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/351213 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/351213 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/351213 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

199: 90


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

199: 91


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# output_dir = '.'
# output_path = os.path.join(output_dir, f"java-test-(train_small).jsonl")
# with open(output_path, "w") as f:
#     for item in data:
#         f.write(json.dumps(item) + "\n")
# print(f"Saved to {output_path}")

Saved to /home/quanvo/Documents/van-vo-projects/llm-code-comment-gen/llm-code-comment-gen/notebooks/java-test-(train_small).jsonl


<b>Step 4: CodeBERT embedding </b>

In [19]:
# dataset = load_dataset("Fsoft-AIC/the-vault-function", 
#                        split_set=["train/small"], 
#                        languages=["java"], 
#                        streaming=False,
#                        trust_remote_code=True)

In [38]:
import numpy as np
import torch
import time
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# codes = [item["code"] for item in dataset["train_small"]]
# batch_size = 32

# # Run through model
# embeddings = []
# i = 0
# print(len(dataset["train_small"]))
# start = time.time()
# for i in range(0, len(codes), batch_size):
#     batch_codes = codes[i:i + batch_size]
#     print(i)
#     inputs = tokenizer(batch_codes, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
#         embeddings.append(embedding.cpu().numpy())
# end = time.time()
# print(end - time)

# embeddings_np = np.array(embeddings, dtype=object)
# embeddings_np = np.vstack(embeddings_np)
# np.save('embedding_func_java_train_small.npy', embeddings_np, allow_pickle=True)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [31]:
import faiss

embeddings_np = np.load('embedding_func_java_train_small.npy', allow_pickle=True)
embeddings_np = np.vstack(embeddings_np)
print(embeddings_np.shape)

(351213, 768)


In [32]:
faiss.normalize_L2(embeddings_np)
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatIP(dimension)

if faiss.get_num_gpus() > 0:
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)

print("indexing...")
index.add(embeddings_np)

indexing...


In [51]:
query_code = data[0]['functions'][5]['code']
query_input = tokenizer(query_code, padding=True, truncation=True, return_tensors="pt").to(device)
query_embedding = model(**query_input).last_hidden_state[:, 0, :].detach().cpu().numpy()

k = 5
distances, indices = index.search(np.array(query_embedding), k)

retrieved_examples = [dataset["train_small"][i.item()]["code"] for i in indices[0]]
print(retrieved_examples[0])

private void startMQTTServer() throws ConfigurationException {
        boolean isServerStarted = false;
        int port;
        if (qpidServiceImpl.getMQTTSSLOnly()) {
            port = qpidServiceImpl.getMqttSSLPort();
        } else {
            port = qpidServiceImpl.getMqttPort();
        }
        if (AndesConfigurationManager.<Boolean>readValue(AndesConfiguration.TRANSPORTS_MQTT_ENABLED)) {
            while (!isServerStarted) {
                Socket socket = null;
                try {
                    InetAddress address = InetAddress.getByName(getMQTTTransportBindAddress());
                    socket = new Socket(address, port);
                    log.info("MQTT Host Address : " + address.getHostAddress() + " Port : " + port);
                    isServerStarted = socket.isConnected();
                    if (isServerStarted) {
                        log.info("Successfully connected to MQTT server on port " + port);
                    }
                } catch (IOE

In [54]:
for i in range(len(data)):
    for j in range(len(data[i]['functions'])):
        print(f"{i}: {j}")
        query_code = data[i]['functions'][j]['code']
        query_input = tokenizer(query_code, padding=True, truncation=True, return_tensors="pt").to(device)
        query_embedding = model(**query_input).last_hidden_state[:, 0, :].detach().cpu().numpy()
        
        k = 5
        distances, indices = index.search(np.array(query_embedding), k)
        retrieved_examples = [{'code': dataset['train_small'][ind.item()]['code'],
                               'docstring': dataset['train_small'][ind.item()]['docstring'],
                               'code_tokens': dataset['train_small'][ind.item()]['code_tokens'],} for ind in indices[0]]
        data[i]['functions'][j]['CodeBERT'] = retrieved_examples

0: 0
0: 1
0: 2
0: 3
0: 4
0: 5
0: 6
0: 7
0: 8
0: 9
0: 10
0: 11
1: 0
1: 1
1: 2
1: 3
1: 4
1: 5
1: 6
1: 7
1: 8
1: 9
1: 10
1: 11
1: 12
1: 13
1: 14
2: 0
2: 1
2: 2
2: 3
2: 4
2: 5
2: 6
2: 7
2: 8
2: 9
2: 10
2: 11
2: 12
2: 13
2: 14
2: 15
2: 16
2: 17
2: 18
2: 19
2: 20
2: 21
2: 22
2: 23
2: 24
2: 25
2: 26
2: 27
2: 28
2: 29
2: 30
2: 31
2: 32
2: 33
2: 34
2: 35
2: 36
2: 37
2: 38
2: 39
2: 40
2: 41
2: 42
2: 43
2: 44
2: 45
2: 46
2: 47
2: 48
2: 49
2: 50
2: 51
2: 52
2: 53
2: 54
2: 55
2: 56
2: 57
2: 58
2: 59
2: 60
2: 61
2: 62
2: 63
2: 64
2: 65
2: 66
2: 67
2: 68
2: 69
2: 70
2: 71
2: 72
2: 73
2: 74
2: 75
2: 76
2: 77
2: 78
2: 79
2: 80
2: 81
2: 82
2: 83
2: 84
2: 85
2: 86
2: 87
2: 88
2: 89
2: 90
2: 91
2: 92
2: 93
2: 94
2: 95
2: 96
2: 97
2: 98
2: 99
2: 100
2: 101
2: 102
2: 103
2: 104
2: 105
2: 106
2: 107
2: 108
2: 109
2: 110
2: 111
2: 112
2: 113
2: 114
2: 115
2: 116
2: 117
2: 118
2: 119
2: 120
2: 121
2: 122
2: 123
2: 124
2: 125
2: 126
2: 127
2: 128
2: 129
2: 130
2: 131
2: 132
2: 133
2: 134
2: 135
2: 136
2: 137
2:

In [55]:
output_dir = '.'
output_path = os.path.join(output_dir, f"java-test-(train_small).jsonl")
with open(output_path, "w") as f:
    for item in data:
        f.write(json.dumps(item) + "\n")
print(f"Saved to {output_path}")

Saved to ./java-test-(train_small).jsonl
